In [90]:
import sys
import os

src_path = r'C:\Users\rshaw\Desktop\EC Utbildning - Data Science\Kurs 9 - Project\Project\ds23_projektkurs\predictive-maintenance\src\Data processing scripts'
sys.path.append(src_path)

from Logging_module import LoggerSetup


### Import Libraries & Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'


### Load pre-processed data from SQL database

In [82]:
data_logger = LoggerSetup(logger_name='Data_Logger', log_file='data_log.log').get_logger()
engine = create_engine('mssql+pyodbc://MSI/predictive_maintenance_db?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')
try:
    data_logger.info("Loading 'PdM_errors' data from the database.")
    errors_df = pd.read_sql('SELECT * FROM PdM_errors', con=engine)
    data_logger.info("'PdM_errors' data loaded successfully.")

    data_logger.info("Loading 'PdM_failures' data from the database.")
    failures_df = pd.read_sql('SELECT * FROM PdM_failures', con=engine)
    data_logger.info("'PdM_failures' data loaded successfully.")

    data_logger.info("Loading 'PdM_machines' data from the database.")
    machines_df = pd.read_sql('SELECT * FROM PdM_machines', con=engine)
    data_logger.info("'PdM_machines' data loaded successfully.")

    data_logger.info("Loading 'PdM_maint' data from the database.")
    maint_df = pd.read_sql('SELECT * FROM PdM_maint', con=engine)
    data_logger.info("'PdM_maint' data loaded successfully.")

    data_logger.info("Loading 'PdM_telemetry' data from the database.")
    telemetry_df = pd.read_sql('SELECT * FROM PdM_telemetry', con=engine)
    data_logger.info("'PdM_telemetry' data loaded successfully.")

except Exception as e:
    data_logger.error(f"Error occurred while loading data: {e}")

### Read datasets

In [ ]:
print("Errors DataFrame:")
display(errors_df.head())

print("Failures DataFrame:")
display(failures_df.head())

print("Machines DataFrame:")
display(machines_df.head())

print("Maintenance DataFrame:")
display(maint_df.head())

print("Telemetry DataFrame:")
display(telemetry_df.head())


In [ ]:
print(telemetry_df.head())
print(telemetry_df['datetime'].min(), telemetry_df['datetime'].max())


In [ ]:
telemetry_df.describe().T

- Here we see sensor data for 4 operational settings that occurred over 12 months between January 1, 2015, and January 01, 2016. 


In [ ]:
print(failures_df.head())
print(failures_df['datetime'].min(), failures_df['datetime'].max())


- Here we see that failures occurred over 12 months between January 2, 2015, and December 31, 2015. - 
- Failures involve different components (comp1, comp2, comp4), all related to machine ID 1 during this time range. 

### EDA

In [ ]:
#Here we drill down to focus on one machine from 100, namely machine no. 17
df_sel_17 = telemetry_df.loc[telemetry_df['machineID'] == 17].reset_index(drop=True)
print(df_sel_17.head(n=5))

In [ ]:
df_sel_17.info()

 - This machine number 17 has 8761 rows and 7 columns, including telemetry data (volt, rotate, pressure, vibration) and machineID.
- All columns are numeric except for the datetime column which is a timestamp, and date which is an object type (string).
- Memory used is approx 479.2 KB 
- There are no missing values.

In [ ]:
print(df_sel_17[['volt', 'rotate', 'pressure', 'vibration']].describe())

- Sensor Ranges: All sensor readings (voltage, rotation, pressure, vibration) show reasonable variability, with voltage and rotation having the highest range of values.

- Relatively Low Variation in Vibration: The vibration sensor has the lowest standard deviation, meaning its values are more tightly clustered around the mean compared to other sensors

In [ ]:
df_sel_17.duplicated().sum()

In [ ]:
round(df_sel_17.isnull().sum() / df_sel_17.isnull().count() * 100, 2)

- This is just an additional check, but we can confirm here that there are no missing values and that our pre-processing of the raw datasets cleared any duplicates.

In [12]:
def histogram_boxplot(df_sel_17, feature, figsize=(12, 7), kde=False, bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (12,7))
    kde: whether to the show density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  
        sharex=True,  
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  
    sns.boxplot(
        data=df_sel_17, x=feature, ax=ax_box2, showmeans=True, color="violet"
    )  
    sns.histplot(
        data=df_sel_17, x=feature, kde=kde, ax=ax_hist2, bins=bins, palette="winter"
    ) if bins else sns.histplot(
        data=df_sel_17, x=feature, kde=kde, ax=ax_hist2
    )  
    ax_hist2.axvline(
        df_sel_17[feature].mean(), color="green", linestyle="--"
    ) 
    ax_hist2.axvline(
        df_sel_17[feature].median(), color="black", linestyle="-"
    )  

In [ ]:
def is_numeric(series):
    return pd.api.types.is_numeric_dtype(series)

for feature in df_sel_17.columns:
    if is_numeric(df_sel_17[feature]) and feature != 'machineID':
        histogram_boxplot(df_sel_17, feature, figsize=(12, 7), kde=False, bins=None)
    else:
        print(f"Skipping column: {feature}")

- Most of the sensors show a fairly normal distribution, with the mean and median values aligning closely.

- Standard Deviation appears to vary across the sensors, with a considerable portion of sensor readings showing spread beyond typical ranges. While some distributions have a higher concentration around the mean, sensors like pressure and voltage show a greater spread with extreme outliers.

- Outliers are present across all four sensors. The voltage and rotation sensors have several outliers on both the lower and upper ends. While these outliers exist, they don't seem to significantly distort the central values (mean and median). The pressure and vibration sensors also show a reasonable number of outliers on the higher end.

- The rotation sensor shows a slight right skew, as shown by a few high-value outliers stretching beyond the bulk of the data. Meanwhile, vibration shows a slight left skew, with more values on the lower side, though it is less pronounced.

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 2, 1)
sns.lineplot(x='datetime', y='volt', data=df_sel_17)
plt.title('Voltage Over Time')

plt.subplot(2, 2, 2)
sns.lineplot(x='datetime', y='rotate', data=df_sel_17)
plt.title('Rotation Over Time')

plt.subplot(2, 2, 3)
sns.lineplot(x='datetime', y='pressure', data=df_sel_17)
plt.title('Pressure Over Time')

plt.subplot(2, 2, 4)
sns.lineplot(x='datetime', y='vibration', data=df_sel_17)
plt.title('Vibration Over Time')

plt.tight_layout()
plt.show()

- There seems to be quite a few spikes that could be anomalies leading to errors or failures.
- We'll investigate further using z-scores to see how many standard deviations a sensor data point is from the mean, where we will target any point outside of -3 and +3 as anomaly.

In [15]:
from sklearn.preprocessing import StandardScaler

#Create plots of Z-scores for this machine to check for anomalies over time in sensor data 
scaler = StandardScaler()
z_scores = scaler.fit_transform(df_sel_17[['volt', 'rotate', 'pressure', 'vibration']])
z_scores_df = pd.DataFrame(z_scores, columns=['volt_zscore', 'rotate_zscore', 'pressure_zscore', 'vibration_zscore'])
df_anomalies = pd.concat([df_sel_17, z_scores_df], axis=1)
anomalies = df_anomalies[
    (df_anomalies['volt_zscore'].abs() > 3) |
    (df_anomalies['rotate_zscore'].abs() > 3) |
    (df_anomalies['pressure_zscore'].abs() > 3) |
    (df_anomalies['vibration_zscore'].abs() > 3)
]

In [ ]:
df_anomalies['datetime'] = pd.to_datetime(df_anomalies['datetime'])
fig, axs = plt.subplots(4, 1, figsize=(12, 16), sharex=True)

axs[0].plot(df_anomalies['datetime'], df_anomalies['volt_zscore'], label='Volt Z-score', color='b')
axs[0].set_ylabel('Volt Z-score')
axs[0].axhline(y=3, color='r', linestyle='--')
axs[0].axhline(y=-3, color='r', linestyle='--')
axs[0].set_title('Volt Z-scores Over Time')

axs[1].plot(df_anomalies['datetime'], df_anomalies['rotate_zscore'], label='Rotate Z-score', color='g')
axs[1].set_ylabel('Rotate Z-score')
axs[1].axhline(y=3, color='r', linestyle='--')
axs[1].axhline(y=-3, color='r', linestyle='--')
axs[1].set_title('Rotate Z-scores Over Time')

axs[2].plot(df_anomalies['datetime'], df_anomalies['pressure_zscore'], label='Pressure Z-score', color='orange')
axs[2].set_ylabel('Pressure Z-score')
axs[2].axhline(y=3, color='r', linestyle='--')
axs[2].axhline(y=-3, color='r', linestyle='--')
axs[2].set_title('Pressure Z-scores Over Time')

axs[3].plot(df_anomalies['datetime'], df_anomalies['vibration_zscore'], label='Vibration Z-score', color='purple')
axs[3].set_ylabel('Vibration Z-score')
axs[3].axhline(y=3, color='r', linestyle='--')
axs[3].axhline(y=-3, color='r', linestyle='--')
axs[3].set_title('Vibration Z-scores Over Time')

axs[3].set_xlabel('Datetime')
plt.tight_layout()
plt.show()

correlation_matrix = df_anomalies[['volt_zscore', 'rotate_zscore', 'pressure_zscore', 'vibration_zscore']].corr()
plt.figure(figsize=(8, 6)) 
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, linewidths=0.5)
plt.title('Correlation Matrix of Z-Scores for Sensors')
plt.show()

- Both pressure and vibration seem to have more frequent spikes exceeding the threshold, indicating more frequent anomalies.

- The most notable correlation is between the rotate_zscore and vibration_zscore (0.63), meaning these sensors tend to show anomalies at the same time. On the other hand, volt_zscore and pressure_zscore have a moderate negative correlation (-0.60), indicating an inverse relationship.

- These anomalies and correlations give us a good reason to zoom into the time period with most visual peaks during during the first quarter of 2015, to investigate potential patterns, causes, and behavior.

In [ ]:
df_anomalies['datetime'] = pd.to_datetime(df_anomalies['datetime'])
start_date = '2015-01-01'
end_date = '2015-03-31'
df_time_window = df_anomalies[(df_anomalies['datetime'] >= start_date) & (df_anomalies['datetime'] <= end_date)]
fig, axs = plt.subplots(4, 1, figsize=(12, 16), sharex=True)

axs[0].plot(df_time_window['datetime'], df_time_window['volt_zscore'], label='Volt Z-score', color='b')
axs[0].set_ylabel('Volt Z-score')
axs[0].axhline(y=3, color='r', linestyle='--')
axs[0].axhline(y=-3, color='r', linestyle='--')
axs[0].set_title('Volt Z-scores Over Time (Filtered Time Window)')

axs[1].plot(df_time_window['datetime'], df_time_window['rotate_zscore'], label='Rotate Z-score', color='g')
axs[1].set_ylabel('Rotate Z-score')
axs[1].axhline(y=3, color='r', linestyle='--')
axs[1].axhline(y=-3, color='r', linestyle='--')
axs[1].set_title('Rotate Z-scores Over Time (Filtered Time Window)')

axs[2].plot(df_time_window['datetime'], df_time_window['pressure_zscore'], label='Pressure Z-score', color='orange')
axs[2].set_ylabel('Pressure Z-score')
axs[2].axhline(y=3, color='r', linestyle='--')
axs[2].axhline(y=-3, color='r', linestyle='--')
axs[2].set_title('Pressure Z-scores Over Time (Filtered Time Window)')

axs[3].plot(df_time_window['datetime'], df_time_window['vibration_zscore'], label='Vibration Z-score', color='purple')
axs[3].set_ylabel('Vibration Z-score')
axs[3].axhline(y=3, color='r', linestyle='--')
axs[3].axhline(y=-3, color='r', linestyle='--')
axs[3].set_title('Vibration Z-scores Over Time (Filtered Time Window)')

axs[3].set_xlabel('Datetime')
plt.tight_layout()
plt.show()


Overall, this first-quarter analysis highlights significant fluctuations, especially in voltage, pressure, and vibration, with several data points indicating potential anomalies or outliers. These might correlate with system irregularities or potential failures if left unchecked.

### Data Preprocessing (in addition to Missis scripts)

In [ ]:
#Here we convert model to an integer to limit features for the model later to avoid multi-collinearity and overfitting.
model_mapping = {value: i for i, value in enumerate(machines_df['model'].unique(), start=1)}
machines_df['model'] = machines_df['model'].map(model_mapping)

data_logger.info(f"Model mapping completed successfully. Mapping: {model_mapping}")
display(machines_df)

In [19]:
# Deal w categorical variables
failures_df = pd.get_dummies(failures_df, columns=['failure'], prefix='fail', dtype=int)
maint_df = pd.get_dummies(maint_df, columns=['comp'], prefix='maint', dtype=int)
errors_df = pd.get_dummies(errors_df, columns=['errorID'], prefix='', prefix_sep='', dtype=int)

In [ ]:
errors_df.rename(columns={'datetime': 'datetime_error'}, inplace=True)
failures_df.rename(columns={'datetime': 'datetime_failure'}, inplace=True)
maint_df.rename(columns={'datetime': 'datetime_maint'}, inplace=True)
telemetry_df.rename(columns={'datetime': 'datetime_telemetry'}, inplace=True)

errors_df['datetime_error'] = pd.to_datetime(errors_df['datetime_error'])
failures_df['datetime_failure'] = pd.to_datetime(failures_df['datetime_failure'])
maint_df['datetime_maint'] = pd.to_datetime(maint_df['datetime_maint'])
telemetry_df['datetime_telemetry'] = pd.to_datetime(telemetry_df['datetime_telemetry'])

print(errors_df['datetime_error'].dtypes)
print(failures_df['datetime_failure'].dtypes)
print(maint_df['datetime_maint'].dtypes)
print(telemetry_df['datetime_telemetry'].dtypes)

In [ ]:
# Create failure, error and maintenance flags to prep for feature engineering
failures_df['failure_flag'] = 1
failure_count = failures_df['failure_flag'].sum()

errors_df['error_flag'] = 1
error_count = errors_df['error_flag'].sum()

maint_df['maint_flag'] = 1
maintenance_count = maint_df['maint_flag'].sum()

print(f"Number of failures: {failure_count}")
print(f"Number of errors: {error_count}")
print(f"Number of maintenances: {maintenance_count}")

In [ ]:
maint_df.head()

In [ ]:
print(telemetry_df.shape)
print(errors_df.shape)
print(failures_df.shape)
print(maint_df.shape)
print(machines_df.shape)

In [ ]:
display(telemetry_df.head())
display(errors_df.head())
display(failures_df.head())
display(maint_df.head())
display(machines_df.head())

In [ ]:
##### Merge Data before feature engineering
telemetry_df = telemetry_df.sort_values('datetime_telemetry')
failures_df = failures_df.sort_values('datetime_failure')
errors_df = errors_df.sort_values('datetime_error')
maintenance_df = maint_df.sort_values('datetime_maint')

merged_df = pd.merge_asof(
    telemetry_df,
    failures_df[['machineID', 'datetime_failure', 'fail_comp1', 'fail_comp2', 'fail_comp3', 'fail_comp4', 'failure_flag']],
    by='machineID',
    left_on='datetime_telemetry',
    right_on='datetime_failure',
    direction='backward',
    tolerance=pd.Timedelta('1d')
)

merged_df = pd.merge_asof(
    merged_df,
    errors_df[['machineID', 'datetime_error', 'error1', 'error2', 'error3', 'error4', 'error5', 'error_flag']],
    by='machineID',
    left_on='datetime_telemetry',
    right_on='datetime_error',
    direction='backward',
    tolerance=pd.Timedelta('1d')
)

merged_df = pd.merge_asof(
    merged_df,
    maintenance_df[['machineID', 'datetime_maint', 'maint_comp1', 'maint_comp2', 'maint_comp3', 'maint_comp4', 'maint_flag']],
    by='machineID',
    left_on='datetime_telemetry',
    right_on='datetime_maint',
    direction='backward',
    tolerance=pd.Timedelta('7d')
)

merged_df = pd.merge(
    merged_df,
    machines_df[['machineID', 'model', 'age']],
    on='machineID',
    how='left'
)

merged_df.fillna(0, inplace=True)

indicator_columns = ['failure_flag', 'error_flag', 'maint_flag', 'fail_comp1', 'fail_comp2', 'fail_comp3', 'fail_comp4', 'error1', 'error2', 'error3', 'error4', 'error5', 'maint_comp1', 'maint_comp2', 'maint_comp3', 'maint_comp4']

for col in indicator_columns:
    if col in merged_df.columns:
        merged_df[col] = merged_df[col].astype(int)

display(merged_df)

In [ ]:
print(merged_df.isnull().sum())

In [ ]:
machine_id_to_plot = 17
plot_df = merged_df[merged_df['machineID'] == machine_id_to_plot]

plt.figure(figsize=(14, 8))
plt.plot(plot_df['datetime_telemetry'], plot_df['pressure'], label='Pressure', color='blue', linewidth=2)
plt.plot(plot_df['datetime_telemetry'], plot_df['vibration'], label='Vibration', color='green', linewidth=2)
if 'failure_flag' in plot_df.columns:
    plt.step(plot_df['datetime_telemetry'], plot_df['failure_flag'] * 50, label='Failure Flag', color='purple', linestyle='--', where='post')
if 'error_flag' in plot_df.columns:
    plt.step(plot_df['datetime_telemetry'], plot_df['error_flag'] * 50, label='Error Flag', color='orange', linestyle='--', where='post')
if 'maint_flag' in plot_df.columns:
    plt.step(plot_df['datetime_telemetry'], plot_df['maint_flag'] * 50, label='Maintenance Flag', color='red', linestyle='--', where='post')
plt.xlabel('Datetime')
plt.ylabel('Telemetry Readings and Event Flags')
plt.title(f'Telemetry Readings and Event Indicators for Machine {machine_id_to_plot}')
plt.legend(loc='upper right')
plt.grid(True)
plt.tight_layout()
plt.show()

- While for only 1 machine, this still shows a trend of reactive mainteance until the last quarter of 2015 and then a major uturn in proactive maintenace at the end of the year. 
- Errors are consistent and can lead to failures but the correlation isn't clear. 

### Feature Engineering 

In [ ]:
# # RUL for machines
from sklearn.preprocessing import MinMaxScaler

# Handling Missing Timestamps and Calculating 'RUL'
merged_df['datetime_failure'] = merged_df['datetime_telemetry'].where(merged_df['failure_flag'] == 1)
merged_df['datetime_failure'] = merged_df.groupby('machineID')['datetime_failure'].bfill()
merged_df['datetime_telemetry'] = pd.to_datetime(merged_df['datetime_telemetry'])
merged_df['datetime_failure'] = pd.to_datetime(merged_df['datetime_failure'], errors='coerce')

num_na_rul = merged_df['datetime_failure'].isna().sum()
print(f"Number of rows with missing failure values (NaT): {num_na_rul}")

merged_df['RUL'] = (merged_df['datetime_failure'] - merged_df['datetime_telemetry']).dt.total_seconds() / 3600

mean_rul = merged_df['RUL'].mean()
print(f"Mean RUL used to impute missing values: {mean_rul}")

merged_df['RUL'].fillna(mean_rul, inplace=True)

rul_data = merged_df['RUL']

In [29]:
# Time since last maintenance for machines (in hours)
merged_df['time_since_last_maint'] = (
    merged_df.groupby('machineID')['datetime_telemetry']
    .transform(lambda x: x - x.where(merged_df['maint_flag'] == 1).ffill())
).dt.total_seconds() / 3600
merged_df['time_since_last_maint'] = merged_df['time_since_last_maint'].fillna(99999)  

In [30]:
# # Time since last failure for components
# # components = ['fail_comp1', 'fail_comp2', 'fail_comp3', 'fail_comp4']
# for component in components:
#     merged_df[f'time_since_last_failure_{component}'] = (
#         merged_df.groupby('machineID')['datetime_telemetry']
#         .transform(lambda x: x - x.where(merged_df[component] == 1).ffill())
#     ).dt.total_seconds() / 3600
#     merged_df[f'time_since_last_failure_{component}'].fillna(0, inplace=True)

In [ ]:
#1 Check RUL values after failure timestamps (should be 0)
df_after_failure = merged_df[merged_df['failure_flag'] == 1]

# Check if any RUL is greater than 0 after failure
incorrect_rul_after_failure = df_after_failure[df_after_failure['RUL'] > 0]

if not incorrect_rul_after_failure.empty:
    print("RUL values after failure that are not 0:")
    display(incorrect_rul_after_failure[['machineID', 'datetime_telemetry', 'failure_flag', 'RUL']])
else:
    print("All RUL values after failure are 0.")


In [ ]:
# Filter rows where failure has not occurred yet (failure_flag == 0)
df_before_failure = merged_df[merged_df['failure_flag'] == 0]
df_before_failure = df_before_failure.sort_values(by=['machineID', 'datetime_telemetry'])

# Calculate the RUL difference for each machine (to check if RUL is decreasing)
df_before_failure['RUL_diff'] = df_before_failure.groupby('machineID')['RUL'].diff()
incorrect_rul_before_failure = df_before_failure[df_before_failure['RUL_diff'] < 0]

# Show rows where RUL is increasing (which shouldn't happen)
if incorrect_rul_before_failure.empty:
    print("RUL values before failure that are not decreasing:")
    display(incorrect_rul_before_failure[['machineID', 'datetime_telemetry', 'RUL', 'RUL_diff']])
else:
    print("All RUL values before failure are decreasing correctly.")

In [ ]:
#3 Check how many rows do not have a scheduled failure (next failure date is in the far future)
merged_df['datetime_failure'] = pd.to_datetime(merged_df['datetime_failure'], errors='coerce') 
merged_df['datetime_next_failure'] = merged_df.groupby('machineID')['datetime_failure'].bfill()

if 'datetime_next_failure' in merged_df.columns:
    print("datetime_next_failure column successfully created.")
else:
    print("datetime_next_failure column is missing.")

future_date = pd.Timestamp('2100-02-02')

no_scheduled_failure = merged_df[merged_df['datetime_next_failure'] == future_date]

num_no_scheduled_failure = len(no_scheduled_failure)
print(f"Number of rows with no scheduled failure: {num_no_scheduled_failure}")

In [36]:
# # Time since last maintenance for components
# for comp in ['maint_comp1', 'maint_comp2', 'maint_comp3', 'maint_comp4']:
#     merged_df[f'time_since_last_{comp}'] = (
#         merged_df.groupby('machineID')['datetime_telemetry']
#         # .transform(lambda x: x - x.where(merged_df[comp] == 1).ffill())
#     ).dt.total_seconds() / 3600  # Convert to hours
#     merged_df[f'time_since_last_{comp}'].fillna(0, inplace=True)

In [ ]:
# Overall maintenance history
overall_maint_hist = merged_df['overall_maintenance_history'] = (
    merged_df.groupby('machineID')['maint_flag']
    .transform('sum')
)
overall_maint_hist

In [38]:
# Interaction features
merged_df['pressure_x_volt'] = merged_df['pressure'] * merged_df['volt']
merged_df['pressure_x_vibration'] = merged_df['pressure'] * merged_df['vibration']
merged_df['rotate_x_vibration'] = merged_df['rotate'] * merged_df['vibration']
merged_df['rotate_x_volt'] = merged_df['rotate'] * merged_df['volt']

# Lag features
merged_df['vibration_t-1'] = merged_df['vibration'].shift(1)
merged_df['pressure_t-1'] = merged_df['pressure'].shift(1)
merged_df['volt_t-1'] = merged_df['volt'].shift(1)
merged_df['rotate_t-1'] = merged_df['rotate'].shift(1)

merged_df.fillna(0, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import numpy as np
import pandas as pd

merged_df_numeric = merged_df.select_dtypes(include=['float64', 'int64', 'int32']).drop(columns=['RUL'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(merged_df_numeric)

isolation_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
isolation_forest.fit(X_scaled)

anomaly_labels = isolation_forest.predict(X_scaled)

merged_df['anomaly_flag'] = (anomaly_labels == -1).astype(int)

print(merged_df[['anomaly_flag']].value_counts())

In [40]:
# Create interaction features between anomalies, failures, and maintenance
merged_df['anomaly_x_failure'] = merged_df['anomaly_flag'] * merged_df['failure_flag']
merged_df['anomaly_x_maint'] = merged_df['anomaly_flag'] * merged_df['maint_flag']


In [ ]:
display(merged_df.head())

In [ ]:
# Ensure all features are numeric before model build
merged_df_numeric = merged_df.select_dtypes(include=['float64', 'int64', 'int32'])
corr_matrix = merged_df_numeric.corr()
merged_df_numeric.columns

In [ ]:
# Plot numeric features 
subset_columns = ['volt', 'rotate', 'pressure', 'vibration', 'pressure_x_volt', 'rotate_x_vibration', 'overall_maintenance_history', 'age']
corr_subset = merged_df_numeric[subset_columns].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr_subset, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation between Telemetry, Interaction Terms, and Maintenance Features')
plt.show()

- Interaction terms like pressure_x_volt and rotate_x_vibration are strongly correlated with their base features (e.g., pressure, volt, vibration), which may indicate redundancy unless they capture non-linear relationships.

- Overall maintenance history and machine age show very weak correlations with sensor data, suggesting these features might not heavily influence real-time telemetry.

- Therefore, we will apply Isolation Forest and Random Forest to help narrow down the feature selection process and identify the most relevant variables for model performance.

In [ ]:
# Plotting the distribution of target feature RUL
plt.figure(figsize=(10, 6))
sns.histplot(rul_data, bins=20, kde=True, color='blue')
plt.title('Distribution of Remaining Useful Life (RUL)', fontsize=16)
plt.xlabel('Remaining Useful Life (RUL)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(True)
plt.show()

- Presence of Imputed RUL Values: The sharp peak around the mean RUL indicates that many machines or components have missing failure information, leading to a concentration of imputed values.
This can skew model training because the imputed RUL values do not represent real failure patterns and could distort model predictions.

- Right-Skewed Distribution: The right-skewed nature of the distribution suggests that most components/machines have a relatively low RUL, with fewer instances of very high RUL values. This is typical in real-world predictive maintenance data, as machines are more likely to be closer to failure than far away from it.

TO DO - Imputation Flag: Add a binary feature that flags rows where the RUL has been imputed. This might help the model distinguish between real and imputed data during prediction.

#### Feature Reduction

In [ ]:
merged_df.columns

In [ ]:
merged_df_numeric.columns

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import pandas as pd

target = 'RUL'
features = merged_df_numeric.drop(columns=[target]).columns.tolist()

train_data = merged_df[(merged_df['datetime_telemetry'] >= '2014-06-01') & 
                       (merged_df['datetime_telemetry'] <= '2015-08-31')]
val_data = merged_df[(merged_df['datetime_telemetry'] >= '2015-09-01') & 
                     (merged_df['datetime_telemetry'] <= '2015-10-31')]
test_data = merged_df[(merged_df['datetime_telemetry'] >= '2015-11-01') & 
                      (merged_df['datetime_telemetry'] <= '2015-12-31')]

X_train, y_train = train_data[features], train_data[target]
X_val, y_val = val_data[features], val_data[target]
X_test, y_test = test_data[features], test_data[target]

rf = RandomForestRegressor(n_estimators=40, n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)

y_val_pred = rf.predict(X_val)
val_mae = mean_absolute_error(y_val, y_val_pred)
print(f"Random Forest Validation MAE: {val_mae}")

y_test_pred = rf.predict(X_test)
test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"Random Forest Test MAE: {test_mae}")

importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances from Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Calculate cumulative importance
feature_importance_df['Cumulative Importance'] = feature_importance_df['Importance'].cumsum()
selected_features = feature_importance_df[feature_importance_df['Cumulative Importance'] <= 0.95]['Feature'].tolist()
print(f"Selected features contributing to 95% of total importance: {selected_features}")

print(len(selected_features))
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(feature_importance_df['Cumulative Importance'], marker='o', linestyle='--')
plt.axhline(y=0.95, color='r', linestyle='-')
plt.title('Cumulative Feature Importance')
plt.xlabel('Number of Features')
plt.ylabel('Cumulative Importance')
plt.show()


In [ ]:
# Features contributing to 95% of total importance
selected_features = feature_importance_df[feature_importance_df['Cumulative Importance'] <= 0.95]['Feature'].tolist()
merged_df_final = merged_df[selected_features]
display(merged_df_final)

In [ ]:
corr_matrix = merged_df_final.corr()

plt.figure(figsize=(20, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merged_df_final)

pca = PCA()
pca.fit(scaled_data)

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_.cumsum(), marker='o', linestyle='--')
plt.title('Cumulative Explained Variance by PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.axhline(y=0.90, color='r', linestyle='-')
plt.grid(True)
plt.show()

explained_variance = pca.explained_variance_ratio_.cumsum()
print("Cumulative Explained Variance:", explained_variance)

- Just under 98% of variance explained by 10 components so we'll use this.

#### Model Preparation

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras 

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_absolute_error

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model  # type: ignore
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Input, Conv1D, Dense, Flatten # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # type: ignore
from tcn import TCN
from sklearn.decomposition import PCA
import warnings

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

train_data = merged_df[(merged_df['datetime_telemetry'] >= '2014-06-01') &
                       (merged_df['datetime_telemetry'] <= '2015-08-31')]
val_data = merged_df[(merged_df['datetime_telemetry'] >= '2015-09-01') &
                     (merged_df['datetime_telemetry'] <= '2015-10-31')]
test_data = merged_df[(merged_df['datetime_telemetry'] >= '2015-11-01') &
                      (merged_df['datetime_telemetry'] <= '2015-12-31')]

train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

reduced_features = merged_df_final.columns.tolist()

train_data_reduced = train_data[reduced_features]
val_data_reduced = val_data[reduced_features]
test_data_reduced = test_data[reduced_features]

num_features = len(reduced_features)

# X (Independent variables)
train_features_scaled = scaler.transform(train_data[reduced_features])
val_features_scaled = scaler.transform(val_data[reduced_features])
test_features_scaled = scaler.transform(test_data[reduced_features])

n_components = 12
pca = PCA(n_components=n_components)

train_features_pca = pca.fit_transform(train_features_scaled)
val_features_pca = pca.transform(val_features_scaled)
test_features_pca = pca.transform(test_features_scaled)

# y (dependent variables)
train_regression_targets = train_data[['RUL']].values
val_regression_targets = val_data[['RUL']].values
test_regression_targets = test_data[['RUL']].values

print(train_features_pca.shape)
print(val_features_pca.shape)
print(test_features_pca.shape)
print(train_regression_targets.shape)
print(val_regression_targets.shape)
print(test_regression_targets.shape)

In [ ]:
import numpy as np

def create_sequences(features, regression_targets, sequence_length):
    """
    Create sequences of features and corresponding regression targets for time series prediction.

    Parameters:
    - features: numpy array of shape (num_samples, num_features), input features.
    - regression_targets: numpy array of shape (num_samples, num_targets), target for regression (e.g., RUL).
    - sequence_length: int, the length of the sequences to create.

    Returns:
    - X: numpy array of shape (num_sequences, sequence_length, num_features), input sequences.
    - y: numpy array of shape (num_sequences, num_targets), regression targets for the last time step of each sequence.
    """
    X = np.array([features[i:i + sequence_length] for i in range(len(features) - sequence_length)])
    y = np.array([regression_targets[i + sequence_length] for i in range(len(features) - sequence_length)])
    return X, y

sequence_length = 72  

X_train, y_train = create_sequences(train_features_pca, train_regression_targets, sequence_length)
X_val, y_val = create_sequences(val_features_pca, val_regression_targets, sequence_length)
X_test, y_test = create_sequences(test_features_pca, test_regression_targets, sequence_length)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("X_train shape:", y_train.shape)
print("X_val shape:", y_val.shape)
print("X_test shape:", y_test.shape)

#### Model Build: Model Architecture and Compile

In [ ]:
import tcn
from tcn import TCN
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

input_shape = (sequence_length, 10) #from PCA reduction
inputs = Input(shape=input_shape)

x = TCN(nb_filters=128, kernel_size=3, dilations=[1, 2, 4, 8, 16],
        nb_stacks=3, padding='causal', kernel_initializer=HeNormal())(inputs)
x = Dropout(0.6)(x)
x = Dense(128, activation='relu')(x)
output_rul = Dense(1, activation='linear', name='regression_output_rul')(x)

model = Model(inputs=inputs, outputs=[output_rul])

model.compile(optimizer=Adam(),
              loss=Huber(),
              metrics=['mean_absolute_error'])
model.summary()

#### Model Preraration & Training

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from sklearn.utils.class_weight import compute_sample_weight

train_data['RUL_clipped'] = np.clip(train_data['RUL'], 0, 4000)

sample_weights = compute_sample_weight('balanced', train_data['RUL_clipped'])

early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00005)

y_train = train_data['RUL_clipped'].values

history = model.fit(
    X_train,
    y_train,
    sample_weight=sample_weights,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr]
)

##### Model Evaluation

##### Model Tuning

##### Model Deployment

#### Key Insights:

In [106]:
import tensorflow as tf
model_path = r'C:\Users\rshaw\Desktop\EC Utbildning - Data Science\Kurs 9 - Project\Project\ds23_projektkurs\predictive-maintenance\models\Trained models'
model_save_path = os.path.join(model_path, 'tcn_multitask_model_v1.keras')
model.save(model_save_path)

In [ ]:
# predictions

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('mssql+pyodbc://MSI/predictive_maintenance_db?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')
table_name = 'robs_merged_PdM_data'
merged_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print(f"Table '{table_name}' has been successfully saved to the database.")

In [ ]:
# table_name = 'robs_processed_PdM_data'

# # Save DataFrame to the SQL database as a new table
# merged_df_final.to_sql(table_name, con=engine, if_exists='replace', index=False)

# print(f"Data saved successfully to '{table_name}' in the database.")